# MIDS-Lecture-10-Spark-IntroductoryExamples

In [2]:
import os
import sys #current as of 9/26/2015
spark_home = os.environ['SPARK_HOME']

if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))
 

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.0
      /_/

Using Python version 2.7.9 (v2.7.9:648dcafa7e5f, Dec 10 2014 10:10:46)
SparkContext available as sc, HiveContext available as sqlContext.


In [20]:
import numpy as np
 

dataRDD = sc.parallelize(np.random.random_sample(1000))   
data2X= dataRDD.map(lambda x: x*2)
dataGreaterThan1 = data2X.filter(lambda x: x > 1.0)
cachedRDD = dataGreaterThan1.cache()

In [21]:
cachedRDD.filter(lambda x: x<1).count()

0

In [22]:
cachedRDD.filter(lambda x: x>1).count()

501

In [23]:
cachedRDD.filter(lambda x: x>1).count()

501

In [24]:
for line in cachedRDD.take(10):
    print line

1.27635884545
1.55510407128
1.37779018564
1.59876656856
1.80699073308
1.67930938099
1.99910019948
1.44956955743
1.000141101
1.12453560506


In [ ]:
#RDD mapper
rdd = sc.parallelize([1,2,3,3])
rdd.map(lambda x:  x + 1).collect()
#returns [2, 3, 4, 4]

In [ ]:
#reduce ACTION
rdd = sc.parallelize([1,2,3,3])
sum = rdd.reduce(lambda x, y: x + y)
# sum.collect()
print sum    #returns 9

In [ ]:
#RDD flatmap ranges of integer ranges
rdd = sc.parallelize([1,2,3,3])
rdd.flatMap(lambda x:  range(x, 8)).collect()
rdd.flatMap(lambda x:  range(x, 10)).distinct().collect()
#returns [8, 1, 9, 2, 3, 4, 5, 6, 7]

In [3]:
#Example 4-1. Creating a pair RDD using the first word as the key in Python

lines = sc.parallelize(["Data line 1", "Mining line 2", "data line 3", "Data line 4", "Data Mining line 5"])
pairs = lines.map(lambda x: (x.split(" ")[0], x))  #first word and the original line
pairs.collect()


[('Data', 'Data line 1'),
 ('Mining', 'Mining line 2'),
 ('data', 'data line 3'),
 ('Data', 'Data line 4'),
 ('Data', 'Data Mining line 5')]

In [4]:
def filterTerm(line):
    if 'data' in line[1]:
        return (line[0])
    
result = pairs.filter(filterTerm)
result.collect()

[('data', 'data line 3')]

In [5]:
result = pairs.filter(lambda keyValue: len(keyValue[1]) < 12)
result.collect()

[('Data', 'Data line 1'), ('data', 'data line 3'), ('Data', 'Data line 4')]

## PAIRED RDD with super efficient word count example

In [3]:
lines = sc.parallelize(["Data line 1", "Mining line 2", "data line 3", "Data line 4", "Data Mining line 5"])
counts = lines.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
        
counts.collect()

[('1', 1),
 ('line', 5),
 ('Mining', 2),
 ('3', 1),
 ('2', 1),
 ('data', 1),
 ('5', 1),
 ('Data', 3),
 ('4', 1)]

In [14]:
# a more efficent word counter (less network communication)
#could hash and do an in-memory combiner at the record level
lines = sc.parallelize(["Data line 1", "Mining line 2", "data line 3", "Data line 4", "Data Mining line 5"])
def emitWordCounts(line):
    wordCounts = []
    for w in line.split(" "):
        wordCounts.append((w, 1))  #could hash and do an in-memory combiner to be more efficient
    return (wordCounts)
    
counts = lines.flatMap(emitWordCounts) #\
             #.reduceByKey(lambda a, b: a + b)
#counts.first()  
counts.collect()

#Level 1 DEBUG Get the mapper to work in isolation  first
#emitWordCounts("Mining line 2")   

#Level 2 debug check the mapper is working
counts = lines.flatMap(emitWordCounts) #\
#             .reduceByKey(lambda a, b: a + b)
counts.collect()

#Level 3 debug check the mapper is working
counts = lines.flatMap(emitWordCounts) \
            .reduceByKey(lambda a, b: a + b)
counts.collect()


[('Data', 1),
 ('line', 1),
 ('1', 1),
 ('Mining', 1),
 ('line', 1),
 ('2', 1),
 ('data', 1),
 ('line', 1),
 ('3', 1),
 ('Data', 1),
 ('line', 1),
 ('4', 1),
 ('Data', 1),
 ('Mining', 1),
 ('line', 1),
 ('5', 1)]

In [15]:
# a more efficent word counter (less network communication)
lines = sc.parallelize(["Data line 1", "Mining line 2", "data line 3", "Data line 4", "Data Mining line 5"])
def emitWordCounts(line):
    wordCounts = []
    for w in line.split(" "):
        wordCounts.append((w, 1))  #could hash and do an in-memory combiner to be more efficient
    return (wordCounts)
    
counts = lines.flatMap(emitWordCounts) \
             .reduceByKey(lambda a, b: a + b)
counts.collect()

[('1', 1),
 ('line', 5),
 ('Mining', 2),
 ('3', 1),
 ('2', 1),
 ('data', 1),
 ('5', 1),
 ('Data', 3),
 ('4', 1)]

In [10]:
# a more efficent word counter (less network communication)
# hash and do an in-memory combiner at the record level
lines = sc.parallelize(["Data line 1", "Mining line 2", "data line 3", "Data line 4", "Data Mining line 5"])
def emitWordCounts(line):
    wordCounts = {}
    for w in line.split(" "):
        if wordCounts.has_key(w):
            wordCounts[w]=wordCounts[w]+1
        else:
            wordCounts[w]=1
    for key, value in  wordCounts.items():
        print key,":", value      
    return (wordCounts.items())  

#Level 1 DEBUG Get the mapper to work in isolation  first
#emitWordCounts("Mining line 2")   

#Level 2 debug check the mapper is working
counts = lines.flatMap(emitWordCounts) #\
#             .reduceByKey(lambda a, b: a + b)
counts.collect()

#Level 3 debug check the mapper is working
counts = lines.flatMap(emitWordCounts) \
            .reduceByKey(lambda a, b: a + b)
counts.collect()

#map partitions

[('1', 1),
 ('data', 1),
 ('5', 1),
 ('Data', 3),
 ('4', 1),
 ('line', 5),
 ('Mining', 2),
 ('3', 1),
 ('2', 1)]

In [ ]:
rdd1 = sc.parallelize({(1, 2), (3, 4), (3, 6)})
rdd2 = sc.parallelize({(3, 9), (3, 6)})
joinedRdd=rdd1.join(rdd2)
#RDD1 = {(1, 2), (3, 4), (3, 6)}
#RDD2 = {(3, 9) (3, 6)}
joinedRdd.collect()

## Example 6-14 in Learn Spark Book. Average with mapPartitions() in Python

In [14]:
def combineCtrs(c1, c2):
    return (c1[0] + c2[0], c1[1] + c2[1])
def basicAvg(nums):
    """Compute the average"""
    nums.map(lambda num: (num, 1)).reduce(combineCtrs)

def partitionCtr(nums):
    """Compute sumCounter for partition"""
    sumCount = [0, 0]
    for num in nums:
        sumCount[0] += num
        sumCount[1] += 1
    return [sumCount]

def fastAvg(nums):
    """Compute the avg"""
    sumCount = nums.mapPartitions(partitionCtr).reduce(combineCtrs)
    return sumCount[0] / float(sumCount[1])

nums = fastAvg(sc.parallelize([1,2,3,4]))
print(nums)

2.5


In [26]:
rdd = sc.parallelize([1,2,3,3]).filter(lambda x: x<3)
rdd.collect()

[1, 2]